## Import Libraries

In [9]:
import sys,os

RES_PATH = 'models' # Folder where all 3D models are kept. Same dir level as this file.

if not os.path.exists(RES_PATH):
    print(f"cannot find {RES_PATH} folder, please update RES_PATH")
    exit(1)
else:
    pass

import pyglet
pyglet.options['shadow_window'] = False

import pyrender
import numpy as np
import trimesh

import matplotlib
import matplotlib.pyplot as plt

from sklearn.neighbors import KDTree

import heapq
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Trimesh Utility

In [3]:
def load_mesh(filename):
    mesh_fp = os.path.join(RES_PATH, filename)
    
    assert os.path.exists(mesh_fp), 'cannot found:'+mesh_fp
    
    t_mesh = trimesh.load(mesh_fp)
    
    return t_mesh

## Mesh Simplification Utility

In [4]:
def calculate_vertex_error_quadric(mesh, vertex_index):
    tri_face_indices = mesh.vertex_faces[vertex_index]
    
    Q = np.zeros((4, 4))
    for i in (i for i in tri_face_indices if i != -1): # Filters out padded -1s.
        face = mesh.faces[i]
        p1 = mesh.vertices[face[0]]
        p2 = mesh.vertices[face[1]]
        p3 = mesh.vertices[face[2]]
        
        # Calculates coefficients a, b, c and d, of the plane equation of the triangle face.
        n = np.cross((p2 - p1), (p3 - p1))
        a, b, c = n = n / np.linalg.norm(n)
        d = np.dot(n, p1)
        
        K_p = np.array([[a**2, a*b, a*c, a*d],
                        [a*b, b**2, b*c, b*d],
                        [a*c, b*c, c**2, c*d],
                        [a*d, b*d, c*d, d**2]])
        Q += K_p
    
    return Q

## Main Code

In [5]:
def simplify_mesh(mesh):
    pass

In [6]:
m = load_mesh("coffee_cup.obj")
Q = calculate_vertex_error_quadric(m, 0)
print(Q)

unable to load materials from: coffee_cup.mtl
specified material (wire_028089177)  not loaded!
[[2.44770672e-04 3.62239863e-02 8.75260338e-04 6.59834934e-01]
 [3.62239863e-02 5.99628633e+00 1.44015075e-01 1.09248900e+02]
 [8.75260338e-04 1.44015075e-01 3.46889730e-03 2.62386663e+00]
 [6.59834934e-01 1.09248900e+02 2.62386663e+00 1.99045323e+03]]


In [35]:
pairs = [(1,2), (3,4), (5,6)]

In [36]:
class PairError:
  def __init__(self, pair, error):
    self.pair = pair
    self.error = error

In [49]:
def compute_optimal_contraction(m, pair, q1, q2):
    q_bar = q1 + q2
    q_bar[3,:] = [0,0,0,1]
    b = np.array([[0],[0],[0],[1]])
    result = np.linalg.solve(q_bar, b)
    return result

In [50]:
def calculate_error(v_bar, pair, q1, q2):
    error = v_bar.T@(q1-q2)@v_bar
    return error[0][0]

In [54]:
def create_heap(m, pairs):
    heap = []
    for pair in pairs:
        q1 = calculate_vertex_error_quadric(m, pair[0])
        q2 = calculate_vertex_error_quadric(m, pair[1])
        v_bar = compute_optimal_contraction(m, pair, q1, q2)
        error = calculate_error(v_bar, pair, q1, q2)
        heapq.heappush(heap, (error, pair))
    return heap

In [60]:
h = create_heap(m, pairs)

(-4.014558650387405, (3, 4))
(0.022367831754548984, (5, 6))
